In [1]:
%%configure -f
{ "conf":{
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

In [2]:
sc.install_pypi_package("pandas")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1618539777806_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from pyspark.sql import functions as F


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Read in datasets

In [4]:
charts = spark.read.parquet('s3://mimic-iii-physionet/parquet/CHARTEVENTS/')
charts = charts.select([F.col(x).alias(x.upper()) for x in charts.columns])
charts

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[ROW_ID: int, SUBJECT_ID: int, HADM_ID: int, ICUSTAY_ID: int, ITEMID: int, CHARTTIME: timestamp, STORETIME: timestamp, CGID: int, VALUE: string, VALUENUM: double, VALUEUOM: string, WARNING: int, ERROR: int, RESULTSTATUS: string, STOPPED: string]

In [5]:
patients = spark.read.parquet('s3://mimic-iii-physionet/parquet/PATIENTS/')
patients = patients.select([F.col(x).alias(x.upper()) for x in patients.columns])
patients

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[ROW_ID: int, SUBJECT_ID: int, GENDER: string, DOB: timestamp, DOD: timestamp, DOD_HOSP: timestamp, DOD_SSN: timestamp, EXPIRE_FLAG: int]

In [6]:
admissions = spark.read.parquet('s3://mimic-iii-physionet/parquet/ADMISSIONS/')
admissions = admissions.select([F.col(x).alias(x.upper()) for x in admissions.columns])
admissions

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[ROW_ID: int, SUBJECT_ID: int, HADM_ID: int, ADMITTIME: timestamp, DISCHTIME: timestamp, DEATHTIME: timestamp, ADMISSION_TYPE: string, ADMISSION_LOCATION: string, DISCHARGE_LOCATION: string, INSURANCE: string, LANGUAGE: string, RELIGION: string, MARITAL_STATUS: string, ETHNICITY: string, EDREGTIME: timestamp, EDOUTTIME: timestamp, DIAGNOSIS: string, HOSPITAL_EXPIRE_FLAG: smallint, HAS_CHARTEVENTS_DATA: smallint]

In [7]:
icustays = spark.read.parquet('s3://mimic-iii-physionet/parquet/ICUSTAYS/')
icustays.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------+-------+----------+--------+--------------+-------------+------------+-----------+-------------------+-------------------+------+
|row_id|subject_id|hadm_id|icustay_id|dbsource|first_careunit|last_careunit|first_wardid|last_wardid|             intime|            outtime|   los|
+------+----------+-------+----------+--------+--------------+-------------+------------+-----------+-------------------+-------------------+------+
|   365|       268| 110404|    280836| carevue|          MICU|         MICU|          52|         52|2198-02-14 23:27:38|2198-02-18 05:26:11| 3.249|
|   366|       269| 106296|    206613| carevue|          MICU|         MICU|          52|         52|2170-11-05 11:05:29|2170-11-08 17:46:57|3.2788|
|   367|       270| 188028|    220345| carevue|           CCU|          CCU|          57|         57|2128-06-24 15:05:20|2128-06-27 12:32:29|2.8939|
|   368|       271| 173727|    249196| carevue|          MICU|         SICU|          52|         23|2120-

In [8]:
from pyspark.sql import functions as F
icustays = icustays.select([F.col(x).alias(x.upper()) for x in icustays.columns])


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
icustays_f = icustays.filter((icustays.FIRST_WARDID == icustays.LAST_WARDID) & (icustays.FIRST_CAREUNIT == icustays.LAST_CAREUNIT) )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Join icu stays with datasets and filter on benchmark

In [10]:
#stays = merge_on_subject_admission(stays, admits)
stays = icustays_f.join(admissions, how='inner', on=['SUBJECT_ID', 'HADM_ID'])
print((stays.count(), len(stays.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(55830, 29)

In [11]:
#stays = merge_on_subject(stays, patients)
stays = stays.join(patients, how='inner', on=['SUBJECT_ID'])
print((stays.count(), len(stays.columns)))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(55830, 36)

In [12]:
import pyspark.sql.functions as func


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
to_keep = stays.groupby(stays.HADM_ID).agg(func.count('ICUSTAY_ID'))
print(to_keep.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

52834

In [14]:
to_keep = to_keep.filter(to_keep['count(ICUSTAY_ID)']==1)
print(to_keep.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

50186

In [15]:
to_keep

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[HADM_ID: int, count(ICUSTAY_ID): bigint]

In [16]:
stays = stays.join(to_keep, how='inner', on=['HADM_ID'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
stays.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

50186

In [18]:
from pyspark.sql.functions import datediff,col

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#stays = add_age_to_icustays(stays)
stays = stays.withColumn("AGE", datediff(col("INTIME"),col("DOB"))/365.25)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
test = stays.withColumn("MORTALITY_INUNIT", F.when((F.col("DOD").isNotNull()) & (F.col("DOD").between(F.col("INTIME"), F.col("OUTTIME"))), 1)
      .when((F.col("DEATHTIME").isNotNull()) & (F.col("DEATHTIME").between(F.col("INTIME"), F.col("OUTTIME"))), 1)
      .otherwise(0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
test2 = test.withColumn("MORTALITY_INHOSPITAL", F.when((F.col("DOD").isNotNull()) & (F.col("DOD").between(F.col("ADMITTIME"), F.col("DISCHTIME"))), 1)
      .when((F.col("DEATHTIME").isNotNull()) & (F.col("DEATHTIME").between(F.col("ADMITTIME"), F.col("DISCHTIME"))), 1)
      .otherwise(0))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
test2.select(test2.MORTALITY_INUNIT, test2.MORTALITY_INHOSPITAL,test2.DOD,test2.ADMITTIME,test2.DISCHTIME,test2.DEATHTIME ).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------------+-------------------+-------------------+-------------------+-------------------+
|MORTALITY_INUNIT|MORTALITY_INHOSPITAL|                DOD|          ADMITTIME|          DISCHTIME|          DEATHTIME|
+----------------+--------------------+-------------------+-------------------+-------------------+-------------------+
|               0|                   0|               null|2149-08-02 00:24:00|2149-08-06 12:24:00|               null|
|               0|                   0|               null|2110-12-16 17:50:00|2110-12-18 17:42:00|               null|
|               0|                   0|               null|2193-04-03 21:30:00|2193-04-07 11:10:00|               null|
|               0|                   0|               null|2149-09-22 13:26:00|2149-09-26 14:27:00|               null|
|               0|                   0|               null|2184-08-20 14:21:00|2184-08-22 15:15:00|               null|
|               0|                   0|2

In [23]:
#stays = filter_icustays_on_age(stays)
test2 = test2.filter(test2.AGE >= 18)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
codes = spark.read.parquet('s3://mimic-iii-physionet/parquet/D_ICD_DIAGNOSES/')
codes = codes.select([F.col(x).alias(x.upper()) for x in codes.columns])
codes

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[ROW_ID: int, ICD9_CODE: string, SHORT_TITLE: string, LONG_TITLE: string]

In [26]:
diag = spark.read.parquet('s3://mimic-iii-physionet/parquet/DIAGNOSES_ICD/')
diag = diag.select([F.col(x).alias(x.upper()) for x in diag.columns])
diag

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[ROW_ID: int, SUBJECT_ID: int, HADM_ID: int, SEQ_NUM: int, ICD9_CODE: string]

In [27]:
diagnoses = diag.join(codes, how='inner', on=['ICD9_CODE'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
filterDF = test2.select(test2.SUBJECT_ID,test2.HADM_ID, test2.ICUSTAY_ID )
filterDF.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

42276

In [29]:
filterDF = test2.select(test2.SUBJECT_ID,test2.HADM_ID, test2.ICUSTAY_ID ).dropDuplicates()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
#diagnoses = filter_diagnoses_on_stays(diagnoses, stays)
diagnoses2 = diagnoses.join(filterDF, how='inner', on=['SUBJECT_ID', 'HADM_ID'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
diagnoses2.select(diagnoses2.ICD9_CODE, diagnoses2.SHORT_TITLE, diagnoses2.LONG_TITLE).dropDuplicates().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

6169

In [33]:
#count_icd_codes(diagnoses, output_path=os.path.join(args.output_path, 'diagnosis_counts.csv'))
codes = diagnoses2.select(diagnoses2.ICD9_CODE, diagnoses2.SHORT_TITLE, diagnoses2.LONG_TITLE).dropDuplicates()
codes_counts = diagnoses2.groupby(diagnoses2.ICD9_CODE).count()
codes = codes.join(codes_counts, how= 'inner', on= ['ICD9_CODE'])
codes = codes.withColumn("count1", col('count'))
codes = codes.filter(codes.count1 > 0).select(codes.ICD9_CODE, codes.SHORT_TITLE, codes.LONG_TITLE, codes.count1).withColumnRenamed("count1", "COUNT")
codes

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[ICD9_CODE: string, SHORT_TITLE: string, LONG_TITLE: string, COUNT: bigint]

In [ ]:
#Get charts data and join with item map to get chart item names

In [35]:
charts = charts.filter(charts.HADM_ID.isNotNull() )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
charts = charts.filter(charts.ICUSTAY_ID.isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
itemid = spark.read.csv('s3://aws-athena-query-results-441066871063-us-east-1/item_map', header=True)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
charts_item = charts.join(itemid.filter(itemid.LINKSTO=='chartevents'), on=['ITEMID'], how='inner').filter(col("VALUE").isNotNull())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Perform chart value cleanup

In [43]:
#clean height
charts_item = charts_item.withColumn('VALUENUM', F.when((charts_item.LEVEL2 == 'Height')
                                                   & ((charts_item.VALUEUOM =='Inch')| (charts_item.VALUEUOM =='In'))
                                                   ,charts_item.VALUENUM* 2.54).otherwise(charts_item.VALUENUM))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
#clean weight
charts_item = charts_item.withColumn('VALUENUM', F.when((charts_item.LEVEL2 == 'Weight')
                                                   & (charts_item['MIMIC LABEL'].contains('oz'))
                                                   ,charts_item.VALUENUM/16)
                                     .when((charts_item.LEVEL2 == 'Weight')
                                                   & (charts_item['MIMIC LABEL'].contains('lb'))
                                                   ,charts_item.VALUENUM* 0.453592)
                                     .otherwise(charts_item.VALUENUM))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
#clean Capillary refill rate
charts_item = charts_item.withColumn('VALUENUM', F.when((charts_item.LEVEL2 == 'Capillary refill rate')
                                                   & (charts_item['VALUE'] == 'Brisk')
                                                   ,0 )
                                     .when((charts_item.LEVEL2 == 'Capillary refill rate')
                                                   & (charts_item['VALUE'] == 'Normal <3 secs')
                                                   ,0)
                                     .when((charts_item.LEVEL2 == 'Capillary refill rate')
                                                   & (charts_item['VALUE'] == 'Abnormal >3 secs')
                                                   ,1)
                                     .when((charts_item.LEVEL2 == 'Capillary refill rate')
                                                   & (charts_item['VALUE'] == 'Delayed')
                                                   ,1)
                                     .otherwise(charts_item.VALUENUM))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
#Clean FIO2
charts_item = charts_item.withColumn('VALUENUM', F.when((charts_item.LEVEL2 == 'Fraction inspired oxygen')
                                                   & (~(charts_item['VALUEUOM'].contains('torr')))
                                                      & (~(charts_item['VALUE'].rlike("[^a-zA-Z\\s+]+")))
                                                   ,charts_item.VALUENUM/100 )
                                                .when((charts_item.LEVEL2 == 'Fraction inspired oxygen')
                                                   & (~(charts_item['VALUEUOM'].contains('torr'))) 
                                                      & (charts_item['VALUE'].cast("int") > 1),
                                                    charts_item.VALUENUM/100)
                                                .otherwise(charts_item.VALUENUM))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
#No need to clean Glucose
charts_item.filter((col('LEVEL2')=='Glucose') & (col('VALUE')=='ERROR') & (col('VALUENUM').isNotNull()) ).orderBy(col("VALUENUM").desc()).select(charts_item['LEVEL2'], charts_item['MIMIC LABEL'],charts_item['VALUENUM'],charts_item['VALUE'], charts_item['VALUEUOM'] ).show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-----------+--------+-----+--------+
|LEVEL2|MIMIC LABEL|VALUENUM|VALUE|VALUEUOM|
+------+-----------+--------+-----+--------+
+------+-----------+--------+-----+--------+

In [48]:
#No need to clean PH
charts_item.filter((col('LEVEL2')=='pH') & (col('VALUE')=='ERROR') & (col('VALUENUM').isNotNull()) ).orderBy(col("VALUENUM").desc()).select(charts_item['LEVEL2'], charts_item['MIMIC LABEL'],charts_item['VALUENUM'],charts_item['VALUE'], charts_item['VALUEUOM'] ).show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+-----------+--------+-----+--------+
|LEVEL2|MIMIC LABEL|VALUENUM|VALUE|VALUEUOM|
+------+-----------+--------+-----+--------+
+------+-----------+--------+-----+--------+

In [49]:
#clean temp
charts_item = charts_item.withColumn('VALUENUM', F.when((charts_item.LEVEL2 == 'Temperature')
                                                   & (charts_item['MIMIC LABEL'].contains('F'))
                                                   ,(charts_item.VALUENUM-32) * (5/9))
                                     .when((charts_item.LEVEL2 == 'Temperature')
                                                   & (charts_item['VALUEUOM'].contains('F'))
                                                   ,(charts_item.VALUENUM-32) * (5/9))
                                     .otherwise(charts_item.VALUENUM))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
#Clean O2 saturation
charts_item = charts_item.withColumn('VALUENUM', F.when((charts_item.LEVEL2 == 'Oxygen saturation')
                                                   & (charts_item['VALUENUM'] <= 1)
                                                   ,charts_item.VALUENUM * 100 )
                                     .otherwise(charts_item.VALUENUM))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Get LOS in hours

In [55]:
test2 = test2.withColumn('LOS_Hours',  (F.col("DISCHTIME").cast("long") - F.col("ADMITTIME").cast("long"))/3600)
test2 = test2.withColumn('LOS_Days',  F.datediff(col("DISCHTIME"),col("ADMITTIME")))
test2.select(test2.LOS_Hours,test2.LOS_Days).show(30)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+--------+
|         LOS_Hours|LOS_Days|
+------------------+--------+
| 85.66666666666667|       4|
| 41.28333333333333|       2|
|154.13333333333333|       6|
|130.36666666666667|       5|
| 335.5833333333333|      14|
|118.16666666666667|       5|
|229.91666666666666|      10|
|176.63333333333333|       7|
|              77.5|       3|
|2.8666666666666667|       0|
|164.43333333333334|       7|
|            213.25|       9|
|21.516666666666666|       1|
|30.016666666666666|       2|
|             143.1|       6|
|             113.0|       5|
| 574.1666666666666|      24|
|28.566666666666666|       1|
|137.06666666666666|       6|
|             159.3|       7|
|             69.05|       3|
| 66.73333333333333|       3|
|            380.05|      16|
| 270.8666666666667|      11|
|             355.0|      15|
| 433.6333333333333|      18|
|            775.65|      32|
| 9.566666666666666|       0|
|142.46666666666667|       6|
|             103.7|       4|
+---------

In [56]:
test3 = test2.select(test2['SUBJECT_ID'],test2['HADM_ID'],test2['ICUSTAY_ID'],test2['ADMITTIME'],test2['DISCHTIME'],test2['LOS_Hours']  ).join(charts_item.select(charts_item['SUBJECT_ID'],charts_item['HADM_ID'],charts_item['ICUSTAY_ID']
                                              ,charts_item['CHARTTIME'],charts_item['LEVEL2'],charts_item['VALUENUM'])
                           , on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID'])
#test3.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Filter for events happening with 6 hours of admission

In [57]:
test3 = test3.withColumn('ADMIT_4',  col("ADMITTIME") + F.expr('INTERVAL 6 HOURS'))
test3.select(test3['ADMITTIME'],test3['ADMIT_4']).show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+
|          ADMITTIME|            ADMIT_4|
+-------------------+-------------------+
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
|2163-11-04 19:45:00|2163-11-05 01:45:00|
+-------------------+-------------

In [92]:
test3.select(test3['SUBJECT_ID'],test3['HADM_ID'],test3['ICUSTAY_ID']).distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

41760

In [58]:
test4 = test3.filter((col('LOS_Hours') > 6.0) & (col("CHARTTIME").between(col("ADMITTIME"), col('ADMIT_4'))))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [60]:
from pyspark.sql.functions import row_number
from pyspark.sql.window import Window

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#get max and min chart events for each value type

In [61]:
w = Window.partitionBy(col("SUBJECT_ID"),col("ICUSTAY_ID"), col("LEVEL2")).orderBy("CHARTTIME")
df_min = test4.withColumn("rn", row_number().over(w)).where(col("rn") == 1).drop(col("rn"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [62]:
w_max = Window.partitionBy(col("SUBJECT_ID"),col("ICUSTAY_ID"), col("LEVEL2")).orderBy(col("CHARTTIME").desc())
df_max = test4.withColumn("rn", row_number().over(w_max)).where(col("rn") == 1).drop(col("rn"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [65]:
df_min = df_min.withColumnRenamed("CHARTTIME","CHARTTIME_min").withColumnRenamed("VALUENUM","VALUENUM_min")
df_max = df_max.withColumnRenamed("CHARTTIME","CHARTTIME_max").withColumnRenamed("VALUENUM","VALUENUM_max")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Join min and max df to get full dataset, 1 row per id

In [67]:
join_features = df_min.join(df_max.select(col("SUBJECT_ID"),col("SUBJECT_ID"),col("HADM_ID"),col("ICUSTAY_ID"),col("LEVEL2")
                                         ,col("CHARTTIME_max"),col("VALUENUM_max")), 
                            on =['SUBJECT_ID','HADM_ID','ICUSTAY_ID','LEVEL2'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#filter for just benchamrk measures

In [69]:
list_of_measures = ['Capillary refill rate','Diastolic blood pressure', 'Fraction inspired oxygen'
                   ,'Glascow coma scale eye opening','Glascow coma scale motor response', 'Glascow coma scale total',
                   'Glascow coma scale verbal response', 'Glucose', 'Heart Rate', 'Height','Mean blood pressure', 'Oxygen saturation'
                   , 'Respiratory rate', 'Systolic blood pressure', 'Temperature', 'Weight', 'pH']

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [70]:
join_features_filter = join_features.filter(col("LEVEL2").isin(list_of_measures))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#change in values feature creation

In [73]:
join_features_filter = join_features_filter.withColumn("percent_change", F.when(col("VALUENUM_min").isNull(), None)
                                                       .when(col("CHARTTIME_min")== col("CHARTTIME_max"), None)
                                                       .otherwise((col("VALUENUM_max")-col("VALUENUM_min"))/col("VALUENUM_min")))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#pivot for final dataset creation

In [82]:
join_features_pivot = join_features_filter.groupBy(col("SUBJECT_ID"), col("HADM_ID"), col("ICUSTAY_ID")).pivot("LEVEL2").agg(F.avg(col("VALUENUM_min")),F.avg(col("percent_change") ))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
join_features_pivot

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SUBJECT_ID: int, HADM_ID: int, ICUSTAY_ID: int, Capillary refill rate_avg(VALUENUM_min): double, Capillary refill rate_avg(percent_change): double, Diastolic blood pressure_avg(VALUENUM_min): double, Diastolic blood pressure_avg(percent_change): double, Fraction inspired oxygen_avg(VALUENUM_min): double, Fraction inspired oxygen_avg(percent_change): double, Glascow coma scale eye opening_avg(VALUENUM_min): double, Glascow coma scale eye opening_avg(percent_change): double, Glascow coma scale motor response_avg(VALUENUM_min): double, Glascow coma scale motor response_avg(percent_change): double, Glascow coma scale total_avg(VALUENUM_min): double, Glascow coma scale total_avg(percent_change): double, Glascow coma scale verbal response_avg(VALUENUM_min): double, Glascow coma scale verbal response_avg(percent_change): double, Glucose_avg(VALUENUM_min): double, Glucose_avg(percent_change): double, Heart Rate_avg(VALUENUM_min): double, Heart Rate_avg(percent_change): double, Height

In [85]:
join_features_pivot = join_features_pivot.select([col(column).alias(column.replace(' ', '_')) for column in join_features_pivot.columns])
join_features_pivot= join_features_pivot.select([col(column).alias(column.replace('avg(', '_')) for column in join_features_pivot.columns])
join_features_pivot= join_features_pivot.select([col(column).alias(column.replace(')', ''))for column in join_features_pivot.columns])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [96]:
join_patient = test2.join(join_features_pivot, on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID'], how='inner')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#categorical data groupings and one-hot encoding

In [120]:
#Set Ethnicity Groups
join_patient=join_patient.withColumn("ethnicity_group",F.when(F.col("ETHNICITY").like("%WHITE%"),"White")
                      .when(F.col("ETHNICITY").like("%ASIAN%"),"Asian")
                      .when(F.col("ETHNICITY").like("%BLACK%"),"Black")
                      .when(F.col("ETHNICITY").like("%HISPANIC%"),"Hispanic")
                      .when(F.col("ETHNICITY").like("%UNKNOWN%"),"Unknown")
                      .when(F.col("ETHNICITY").like("%UNABLE%"),"Unknown")
                      .when(F.col("ETHNICITY").like("%DECLINED%"),"Unknown")
                      .otherwise("Other_Eth"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [123]:
join_patient_eth = join_patient.groupBy(col("SUBJECT_ID"), col("HADM_ID"), col("ICUSTAY_ID")).pivot("ethnicity_group").agg(F.lit(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [104]:
join_patient_eth

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[SUBJECT_ID: int, HADM_ID: int, ICUSTAY_ID: int, Asian: int, Black: int, Hispanic: int, Other: int, Unknown: int, White: int]

In [129]:
#Set Religion groups
join_patient=join_patient.withColumn("religion_group",F.when(F.col("RELIGION").like("%NOT%"),"Unknown_Rel")
                      .when(F.col("RELIGION").like("%UNOBTAINABLE%"),"Unknown_Rel")
                      .when(F.col("RELIGION").like("%CATHOLIC%"),"Catholic")
                      .when(F.col("RELIGION").like("%PROTESTANT%"),"ProtestantQuaker")
                      .when(F.col("RELIGION").like("%JEWISH%"),"Jewish")
                      .when(F.col("RELIGION").like("%EPISCOPALIAN%"),"Episcopalian")
                      .otherwise("Other_Rel"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [130]:
join_patient_religion = join_patient.groupBy(col("SUBJECT_ID"), col("HADM_ID"), col("ICUSTAY_ID")).pivot("religion_group").agg(F.lit(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [131]:
#Set Marital Status groups
join_patient=join_patient.withColumn("marital_status_group",F.when(F.col("MARITAL_STATUS").like("%MARRIED%"),"Married")
                      .when(F.col("MARITAL_STATUS").like("%SINGLE%"),"Single")
                      .when(F.col("MARITAL_STATUS").like("%WIDOW%"),"Widowed")
                      .when(F.col("MARITAL_STATUS").like("%DIVORCE%"),"Divorced")
                      .when(F.col("MARITAL_STATUS").like("%SEPARATE%"),"Separated")
                      .otherwise("Other_Mar"))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [125]:
join_patient_mar = join_patient.groupBy(col("SUBJECT_ID"), col("HADM_ID"), col("ICUSTAY_ID")).pivot("marital_status_group").agg(F.lit(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
join_patient_gender = join_patient.groupBy(col("SUBJECT_ID"), col("HADM_ID"), col("ICUSTAY_ID")).pivot("GENDER").agg(F.lit(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [110]:
join_patient_insurance = join_patient.groupBy(col("SUBJECT_ID"), col("HADM_ID"), col("ICUSTAY_ID")).pivot("INSURANCE").agg(F.lit(1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [132]:
all_columns = join_patient.join(join_patient_eth, on= ['SUBJECT_ID','HADM_ID','ICUSTAY_ID'], how='left')
all_columns = all_columns.join(join_patient_religion,  on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID'], how='left')
all_columns = all_columns.join(join_patient_mar,  on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID'], how='left')
all_columns = all_columns.join(join_patient_gender,  on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID'], how='left')
all_columns = all_columns.join(join_patient_insurance,  on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID'], how='left')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [134]:
all_columns2 = all_columns.drop( 'ROW_ID', 'DBSOURCE', 'FIRST_CAREUNIT', 'LAST_CAREUNIT', 'FIRST_WARDID', 'LAST_WARDID', 'INTIME', 'OUTTIME','DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION', 
                               'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME','HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA', 'ROW_ID', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN', 
                               'EXPIRE_FLAG', 'count(ICUSTAY_ID)', 'MORTALITY_INUNIT', 'MORTALITY_INHOSPITAL', 'LOS_Hours', 'LOS_Days', 'ethnicity_group', 
                                'religion_group', 'marital_status_group' )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
#Write final dataset to s3 bucket to get in python notebook

In [138]:
all_columns2.write.format("csv").option("header","true").mode("Overwrite").save("s3://aws-athena-query-results-441066871063-us-east-1/all_columns2")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…